In [16]:
import numpy as np
import pandas as pd

In [17]:
def convert_to_index(homes, locations):
    new_homes = np.zeros(homes.shape, dtype=int)
    for i in range(len(locations)):
        indices = np.argwhere(homes == locations[i])
        np.put(new_homes, indices, i)
    return new_homes

In [18]:
def load(path):
    data = np.loadtxt(path, dtype=str, skiprows=5)
    data = np.where(data == 'x', np.inf, data)
    data = np.array(data, dtype=float)
    
    locations = np.loadtxt(path, dtype=str, skiprows=2, max_rows=1)
    
    homes = np.loadtxt(path, dtype=str, skiprows=3, max_rows=1)
    new_homes = convert_to_index(homes, locations)
    
    return data, locations, new_homes
    

In [19]:
def all_pair_distance(data):
    dist = np.copy(data)
    for k in range(data.shape[0]):
        for i in range(data.shape[0]):
            for j in range(data.shape[0]):
                if i == j:
                    dist[i][j] = 0
                else:
                    dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
    return dist

In [20]:
def initializ_centroids(dist, homes):
    np.random.RandomState(123)
    random_idx = np.random.permutation(homes)
    centroids = dist[random_idx[:10]]
    
    return centroids

In [21]:
def find_closest_cluster(centroids):
    clusters = np.argmin(centroids, axis=0)
    return clusters

In [22]:
def compute_distance(dist, centroids):
    centers = np.argmin(centroids, axis=1).flatten()
    new_distance = dist[centers]
    return new_distance, centers

In [23]:
def compute_centroids(dist, clusters, n_clusters, homes):
    centroids = np.zeros((n_clusters, dist.shape[0]))
    for k in range(n_clusters):
        tmp = np.argwhere(clusters == k).flatten()
        tmp = tmp[np.in1d(tmp, homes)]
        centroids[k, :] = np.mean(dist[tmp], axis=0)
    return centroids

In [25]:
def translate(clusters, random_idx):
    new_clusters = np.zeros(clusters.shape, dtype=int)
    for i in range(len(random_idx[:10])):
        indices = np.argwhere(clusters == i)
        np.put(new_clusters, indices, random_idx[i])
    return new_clusters

In [26]:
def fit(dist, homes):
    centroids = initializ_centroids(dist, homes)
    for i in range(max_iter):
        old_centroids = centroids
        distance, centers = compute_distance(dist, old_centroids)
        clusters = find_closest_cluster(distance)
        centroids = compute_centroids(dist, clusters, n_clusters, homes)
        if np.all(old_centroids == centroids):
            break
        new_clusters = translate(clusters, centers)
    return np.argmin(centroids, axis=1), new_clusters

In [ ]:
def sanity_check(dist_matrix, clusters, centers, starting_location):
    sd = np.std(dist_matrix)
    mean = np.mean(dist_matrix)
    check = np.zeros(dist_matrix.shape[0])
    for i in range(dist_matrix.shape[0]):
        check[i] = dist_matrix[i][clusters[i]]
    for i in range(len(check)):
        if (check[i] > mean + 2.5*sd):
            clusters[i] = i
            centers = np.concatenate((centers, [i]), axis=0)
    if starting_location not in centers:
        centers = np.concatenate((centers, [starting_location]), axis=0)
        clusters[starting_location] = starting_location
    return centers, clusters

In [125]:
V = n_clusters
from sys import maxsize 
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

# implementation of traveling Salesman Problem 
def travellingSalesmanProblem(graph, s, path): 
    # store all vertex apart from source vertex 
    vertex = [] 
    tmp = path
    for i in range(graph.shape[0]): 
        if i != s: 
            vertex.append(i) 
    # store minimum weight Hamiltonian Cycle 
    min_path = maxsize 
    while True: 
        # store current Path weight(cost) 
        current_pathweight = 0
        # compute current path weight 
        k = s 
        for i in range(len(vertex)): 
            current_pathweight += graph[k][vertex[i]] 
            path.append(k)
            k = vertex[i] 
        path.append(k)
        current_pathweight += graph[k][s] 
        # update minimum 
        if min_path > current_pathweight:
            min_path = current_pathweight
            tmp = path
        path = [] 
        if not next_permutation(vertex): 
            break
    return min_path, tmp

# next_permutation implementation 
def next_permutation(L): 
    n = len(L) 
    i = n - 2
    while i >= 0 and L[i] >= L[i + 1]: 
        i -= 1
    if i == -1: 
        return False
    j = i + 1
    while j < n and L[j] > L[i]: 
        j += 1
    j -= 1
    L[i], L[j] = L[j], L[i] 
    left = i + 1
    right = n - 1
    while left < right: 
        L[left], L[right] = L[right], L[left] 
        left += 1
        right -= 1
    return True

In [29]:
def dijsktra(dist, initial, end, homes):
    shortest_paths = {initial: (None, 0)}
    current_node = initial
    visited = set()
    
    while current_node != end:
        visited.add(current_node)
        weight_to_current_node = shortest_paths[current_node][1]

        for next_node in range(len(dist)):
            weight = weight_to_current_node + dist[current_node][next_node]
            if next_node not in shortest_paths:
                shortest_paths[next_node] = (current_node, weight)
            else:
                current_shortest_weight = shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    shortest_paths[next_node] = (current_node, weight)
        
        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
        
        for node in next_destinations:
            if node in homes:
                a, b = next_destinations[node]
                next_destinations[node] = (a, 1/3*b)
        
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
    
    # Work back through destinations in shortest path
    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_paths[current_node][0]
        current_node = next_node
    # Reverse path
    path = path[::-1]
    return path
    

In [165]:
path = "./inputs/43_200.in"
data, locations, homes = load(path)
max_iter = 50
n_clusters = 10

In [166]:
dist_matrix = all_pair_distance(data)

In [167]:
data

array([[ inf,  inf,  10., ...,  inf,  inf, 200.],
       [ inf,  inf,  inf, ...,  inf,  inf,  inf],
       [ 10.,  inf,  inf, ...,  inf,  inf,  inf],
       ...,
       [ inf,  inf,  inf, ...,  inf,  18.,  10.],
       [ inf,  inf,  inf, ...,  18.,  inf,  11.],
       [200.,  inf,  inf, ...,  10.,  11.,  inf]])

In [168]:
dist_matrix

array([[  0., 206.,  10., ..., 210., 211., 200.],
       [206.,   0., 216., ..., 416., 417., 406.],
       [ 10., 216.,   0., ..., 220., 221., 210.],
       ...,
       [210., 416., 220., ...,   0.,  18.,  10.],
       [211., 417., 221., ...,  18.,   0.,  11.],
       [200., 406., 210., ...,  10.,  11.,   0.]])

In [169]:
centers, clusters = fit(dist_matrix, homes)
s = 0

In [170]:
centers

array([154,  68,  87,  35, 129, 185,  79,  10, 104,  60])

In [171]:
centers, clusters = sanity_check(dist_matrix, clusters, centers, s)

In [172]:
centers

array([154,  68,  87,  35, 129, 185,  79,  10, 104,  60,  18,  43,  93,
       118, 143, 168, 193,   0])

In [173]:
dist_matrix[centers, :].shape

(18, 200)

In [174]:
graph = dist_matrix[centers, :][:, centers]

In [175]:
graph

array([[0.00000e+00, 7.08350e+04, 5.53000e+02, 8.21000e+02, 1.93000e+02,
        2.64000e+02, 5.61000e+02, 5.80000e+02, 4.34000e+02, 7.79000e+02,
        7.06600e+04, 7.09010e+04, 7.05940e+04, 7.04670e+04, 7.02260e+04,
        7.01030e+04, 7.03440e+04, 5.45000e+02],
       [7.08350e+04, 0.00000e+00, 7.03120e+04, 7.02730e+04, 7.06420e+04,
        7.08300e+04, 7.02740e+04, 7.05140e+04, 7.04010e+04, 7.00800e+04,
        1.40570e+05, 1.40329e+05, 1.40377e+05, 1.40504e+05, 1.40745e+05,
        1.40938e+05, 1.40886e+05, 7.05490e+04],
       [5.53000e+02, 7.03120e+04, 0.00000e+00, 4.49000e+02, 3.60000e+02,
        8.17000e+02, 3.80000e+01, 6.90000e+02, 1.19000e+02, 2.56000e+02,
        7.07460e+04, 7.05050e+04, 7.00950e+04, 7.02220e+04, 7.04630e+04,
        7.06560e+04, 7.08970e+04, 7.25000e+02],
       [8.21000e+02, 7.02730e+04, 4.49000e+02, 0.00000e+00, 7.79000e+02,
        5.57000e+02, 4.11000e+02, 2.41000e+02, 5.38000e+02, 1.93000e+02,
        7.02970e+04, 7.00800e+04, 7.05140e+04, 7.0641

In [ ]:
tsp_path = []
min_path, tsp_path = travellingSalesmanProblem(graph, 0, tsp_path)
tsp_path

In [ ]:
tsp_path = []
min_path, tsp_path = travellingSalesmanProblem(graph, 0, tsp_path)
for i in range(len(tsp_path)):
    tsp_path[i] = centers[tsp_path[i]]
# tsp_path = np.roll(tsp_path, -tsp_path.index(s))
tsp_path = np.concatenate((tsp_path, [tsp_path[0]]))
print(tsp_path)

In [ ]:
# find the real path using dijsktra
real_path = []
for i in range(len(tsp_path) - 1):
    real_path = real_path + dijsktra(data, tsp_path[i], tsp_path[i + 1], homes)[:-1]
real_path = real_path + [tsp_path[0]]
print(real_path)

In [103]:
# construct the drop_off_mapping
helper = dist_matrix[real_path]
mins = np.argmin(helper, axis=0)
result = np.zeros(mins.shape)
for i in range(len(real_path)):
    indices = np.argwhere(mins == i)
    np.put(result, indices, real_path[i])
for i in range(len(result)):
    if i not in homes:
        result[i] = -1
drop_off_mapping = {}
for i in range(len(real_path)):
    drop_off_mapping[real_path[i]] = np.argwhere(result == real_path[i]).flatten().tolist()
    if not drop_off_mapping[real_path[i]]:
        del drop_off_mapping[real_path[i]]

In [104]:
drop_off_mapping

{0: [2,
  5,
  10,
  17,
  20,
  23,
  26,
  29,
  35,
  40,
  45,
  48,
  49,
  51,
  55,
  61,
  66,
  68,
  72,
  75,
  78,
  104,
  113,
  118,
  120,
  123,
  126,
  127,
  129,
  162,
  165,
  173,
  176,
  184,
  187,
  190,
  192,
  196],
 8: [8],
 14: [14],
 32: [32],
 71: [71],
 107: [107],
 124: [124],
 156: [156],
 159: [159],
 179: [179]}

In [ ]:
path = "./inputs/43_200.in"
data, locations, homes = load(path)
dist_matrix = all_pair_distance(data)
centers, clusters = fit(dist_matrix, homes)
clusters, centers = sanity_check(dist_matrix, clusters, centers)

In [ ]:
graph = dist_matrix[centers, :][:, centers]
s = 0
path = []
# print(travellingSalesmanProblem(graph, s, path)) 
min_path, path = travellingSalesmanProblem(graph, s, path)
path.append(s)
# print(min_path, path)
for i in range(len(path)):
    path[i] = centers[path[i]]

In [576]:
final = []
for i in range(len(path) - 1):
    final = final + dijsktra(data, path[i], path[i + 1], homes)[:-1]
final = final + [path[0]]

In [616]:
helper = dist_matrix[final]
mins = np.argmin(helper, axis=0)
result = np.zeros(mins.shape)
for i in range(len(final)):
    indices = np.argwhere(mins == i)
    np.put(result, indices, final[i])
for i in range(len(result)):
    if i not in homes:
        result[i] = -1

In [632]:
drop_off_mapping = {}
for i in range(len(final)):
    drop_off_mapping[final[i]] = np.argwhere(result == final[i]).flatten().tolist()
    if not drop_off_mapping[final[i]]:
        del drop_off_mapping[final[i]]

In [468]:
# def plot():
#     A = graph
#     G = nx.from_numpy_matrix(np.array(A)) 
#     edge_labels = dict( ((u, v), d["weight"]) for u, v, d in G.edges(data=True) )
#     pos = nx.spr``ing_layout(G)
#     nx.draw(G, pos, with_labels = True)
#     nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
#     plt.show()
# plot()

[159, 113, 39, 70, 89, 13, 61, 66, 129, 28, 159]